In [ ]:
# Load necessary modules and libraries

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, Matern, ExpSineSquared,DotProduct

import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Load the data


Geometry1 = pd.read_csv('Surface_features.csv',header=0, usecols=(4,8,9,10,11,12,14))
Geometry = pd.read_csv('Surface_features.csv',header=0, usecols=(4,6,7,8,9,10,11,12)).values

Ra_ch = pd.read_csv('Surface_features.csv',header=0,usecols=(5,)).values
Ra_ch = Ra_ch[:,0]
ks = pd.read_csv('Surface_features.csv',header=0,usecols=(13,)).values
ks = ks[:,0]



Geometry1["ks"]= np.divide(ks,Ra_ch)
Geometry1["krms_ch"]= np.divide(Geometry1["krms_ch"],Ra_ch)


Geometry1.rename({'krms_ch': '$k_{rms}/R_a$',
                  'pro_ch': '$P_o$',
                  'ESx_ch': '$E_x$',
                  'ESz_ch': '$E_z$',
                  'sk_ch': '$S_k$',
                  'ku_ch': '$K_u$',
                  'ks': '$k_s/R_a$',
                  'label': 'Label',
                  }, axis='columns', errors="raise",inplace = True)


In [ ]:
# Plot raw data

plt.rc('text', usetex=True)

sns.set(context='paper',
            style='ticks', 
            palette='deep',
            font='sans-serif', 
            font_scale=3, color_codes=True, rc=None)


g = sns.pairplot(Geometry1,diag_kind="kde", #palette="seismic", 
                 hue='Label',
                 plot_kws=dict(s=70,facecolor="w", edgecolor="w", linewidth=1),
                 diag_kws=dict(linewidth=1.5))
g.map_upper(sns.kdeplot)
g.map_lower(sns.scatterplot, s=50,)

plt.savefig('pair.pdf', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)



In [ ]:
# Data reconfiguration, to be used in ML

X = Geometry
y = np.divide(ks,Ra_ch)


X[:,0] = np.divide(X[:,0],Ra_ch)
X[:,2] = np.abs(X[:,2])


# Generate secondary features and append them to the original dataset

n,m = X.shape 
X0 = np.ones((n,1))
X1 = np.ones((n,1))
X2 = np.ones((n,1))
X3 = np.ones((n,1))
X4 = np.ones((n,1))
X5 = np.ones((n,1))
X6 = np.ones((n,1))
X7 = np.ones((n,1))
X8 = np.ones((n,1))
X9 = np.ones((n,1))




X1[:,0] = np.transpose(X[:,4]*X[:,5])
X2[:,0] = np.transpose(X[:,4]*X[:,6])
X3[:,0] = np.transpose(X[:,4]*X[:,7])
X4[:,0] = np.transpose(X[:,5]*X[:,6])
X5[:,0] = np.transpose(X[:,5]*X[:,7])
X6[:,0] = np.transpose(X[:,6]*X[:,7])
X7[:,0] = np.transpose(X[:,4]*X[:,4])
X8[:,0] = np.transpose(X[:,5]*X[:,5])
X9[:,0] = np.transpose(X[:,6]*X[:,6])




X = np.hstack((X,X1))
X = np.hstack((X,X2))
X = np.hstack((X,X3))
X = np.hstack((X,X4))
X = np.hstack((X,X5))
X = np.hstack((X,X6))
X = np.hstack((X,X7))
X = np.hstack((X,X8))
X = np.hstack((X,X9))




In [ ]:
# Best linear estimation


reg = LinearRegression().fit(X, y)
reg.score(X, y)
yn=reg.predict(X)
print("Mean err: %f" % np.mean(100.*abs(yn-y)/(y)))
print("Max err: %f" % max(100.*abs(yn-y)/(y)))





In [ ]:
# Define two files that store the best ML prediction based on either L1 or L_\infty norms


filename1 = 'GPR_Linf.sav'
filename2 = 'GPR_L1.sav'





In [ ]:
# Perform ML training --- it may take some time. 
# Adjust ranges for by4 for faster (but potentially less accurate) results.




miny1=100
miny2=100
by4=0.
while by4<10000.:
            by4=by4+1
            kernel1 = RBF(10, (1e-3, 1e2))
            kernel2 = RBF(5, (1e-3, 1e2))
            kernel3 = RationalQuadratic(length_scale=1.0, alpha=0.1)
            kernel4 = Matern(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0), nu=4.5)
            kernel5 = ExpSineSquared(length_scale=2.0, 
                        periodicity=3.0, 
                        length_scale_bounds=(1e-05, 100000.0), 
                        periodicity_bounds=(1e-05, 100000.0))
            kernel6 = DotProduct()
            
            gpr = GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=1000)
            gpr = GaussianProcessRegressor(kernel=kernel3, n_restarts_optimizer=1000,alpha=.1)
            
            
            
            print("by4: %f" % by4)

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
            gpr.fit(X_train, y_train)
            yn, sigma = gpr.predict(X, return_std=True)
                
            #print("Max err: %f" % max(100.*abs(yn-y)/y))
            #print("Mean err: %f" % np.mean(100.*abs(yn-y)/y))
                
            if miny1>max(100.*abs(yn-y)/y):
                    pickle.dump(gpr, open(filename1, 'wb'))
                    miny1=max(100.*abs(yn-y)/y)
                    print("Miny1: %f" % miny1)
                
            if miny2>np.mean(100.*abs(yn-y)/y):
                    pickle.dump(gpr, open(filename2, 'wb'))
                    miny2=np.mean(100.*abs(yn-y)/y)
                    print("Miny2: %f" % miny2)
    

    
    
print("by4: %f" % by4)



In [ ]:
# Load either file1 or file2 to extract the results

loaded_model = pickle.load(open(filename2, 'rb'))
loaded_model.get_params()


In [ ]:

yn, sigma = loaded_model.predict(X,return_std=True)
print("PREDICTED k_s/R_a= ")
print(yn)
print("Max err: %f" % max(100.*abs(yn-y)/(y)))
print("mean err: %f" % np.mean(100.*abs(yn-y)/(y)))  


Error=pd.DataFrame()
Error["$k_s/Ra$"]= y
Error["$k_{sp}/Ra$"]= yn
Error["$error(\%)$"]= (100.*(yn-y)/(y))
Error["Label"]= Geometry1["Label"]

print(Error)

In [ ]:
# Plot the results


plt.rc('text', usetex=True)

sns.set(context='paper',
            style='ticks', 
            palette='deep',
            font='sans-serif', 
            font_scale=2, color_codes=True, rc=None)

g = sns.pairplot(Error,diag_kind="kde", hue='Label',
                 aspect=1.,
                 plot_kws=dict(s=50,facecolor="w", edgecolor="w", linewidth=1.),
                 diag_kws=dict(linewidth=1.5,kernel='gau'))

g.map_upper(sns.kdeplot)
g.map_lower(sns.scatterplot, s=50,legend='full')
g.axes[-2,0].plot(range(15), range(15),'k--', linewidth= 1.7)


for i in range(0,3):
    for ax in g.axes[:,i]:
        ax.spines['top'].set_visible(True)
        ax.spines['right'].set_visible(True)




plt.savefig('GPR_result.pdf', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)



In [ ]:
# Plot confidence interval

sns.set(context='notebook',
            style='ticks', 
            palette='seismic',
            font='sans-serif', 
            font_scale=5, color_codes=True, rc=None)
plt.rc('text', usetex=True)
fig = plt.figure(figsize=(50,55))

plt.subplot(411)
Xm=X[np.argsort(X[:,0])]
Xm=Xm[:,0]
ym=y[np.argsort(X[:,0])]
ymp=yn[np.argsort(X[:,0])]
sigmap=sigma[np.argsort(X[:,0])]

plt.plot(Xm, ym, 'r.', markersize=26)
plt.plot(Xm, ymp, 'b-',linewidth=6)
plt.fill(np.concatenate([Xm, Xm[::-1]]),
         np.concatenate([ymp - 1.900 * sigmap,
                        (ymp + 1.900 * sigmap)[::-1]]),
         alpha=.5, fc='b', ec='None')
plt.xlabel('$k_{rms}/R_a$')
plt.ylabel('$k_s/R_a$')
plt.grid(alpha=0.15)
#plt.legend(loc='best')


plt.subplot(412)
Xm=X[np.argsort(X[:,4])]
Xm=Xm[:,4]
ym=y[np.argsort(X[:,4])]
ymp=yn[np.argsort(X[:,4])]
sigmap=sigma[np.argsort(X[:,4])]

plt.plot(Xm, ym, 'r.', markersize=26)
plt.plot(Xm, ymp, 'b-',linewidth=6)
plt.fill(np.concatenate([Xm, Xm[::-1]]),
         np.concatenate([ymp - 1.900 * sigmap,
                        (ymp + 1.900 * sigmap)[::-1]]),
         alpha=.5, fc='b', ec='None')
plt.xlabel('$E_x$')
plt.ylabel('$k_s/R_a$')
plt.grid(alpha=0.15)


plt.subplot(413)
Xm=X[np.argsort(X[:,3])]
Xm=Xm[:,3]
ym=y[np.argsort(X[:,3])]
ymp=yn[np.argsort(X[:,3])]
sigmap=sigma[np.argsort(X[:,3])]

plt.plot(Xm, ym, 'r.', markersize=26)
plt.plot(Xm, ymp, 'b-',linewidth=6)
plt.fill(np.concatenate([Xm, Xm[::-1]]),
         np.concatenate([ymp - 1.900 * sigmap,
                        (ymp + 1.900 * sigmap)[::-1]]),
         alpha=.5, fc='b', ec='None')
plt.xlabel('$P_o$')
plt.ylabel('$k_s/R_a$')
plt.grid(alpha=0.15)

plt.subplot(414)
Xm=X[np.argsort(X[:,6])]
Xm=Xm[:,6]
ym=y[np.argsort(X[:,6])]
ymp=yn[np.argsort(X[:,6])]
sigmap=sigma[np.argsort(X[:,6])]

plt.plot(Xm, ym, 'r.', markersize=26, label='$k_s/R_a$')
plt.plot(Xm, ymp, 'b-', linewidth=6,label='$k_{sp}/R_a$')
plt.fill(np.concatenate([Xm, Xm[::-1]]),
         np.concatenate([ymp - 1.900 * sigmap,
                        (ymp + 1.900 * sigmap)[::-1]]),
         alpha=.5, fc='b', ec='None', label='$90\%$ $CI$')
plt.xlabel('$S_k$')
plt.ylabel('$k_s/R_a$')
plt.grid(alpha=0.15)
plt.legend(loc='best')


plt.savefig('GPR_CI.pdf', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

